In [2]:
import numpy as np 
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import os

# Asignar la ruta donde deseas guardar las imágenes
PROJECT_ROOT_DIR = r"C:\Users\ezequ\OneDrive\Documentos\Facultad\Optimizacion\tp1"
CHAPTER_ID = "imagenes"  # El nombre de la capeta donde voy a guardar las imagenes

# Crear la ruta completa donde se guardarán las imágenes
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)



In [5]:
path = r'C:\Users\ezequ\OneDrive\Documentos\Facultad\Aprendizaje Automatico\material-main\tp\01_aprendizaje_supervisado\datos\data.csv'
#df = pd.read_csv(path)
df = pd.read_csv('data/data.csv')

In [6]:
df['target'].value_counts()

target
0    343
1    157
Name: count, dtype: int64

La relacion entre clases es 157/343 = 0.46 siendo la mayoritaria la clase representada por el 0 por lo que estamos ante un dataset desbalanceado

A priori hacer un split 400 100 para desarrollo validacion suena razonable , estamos dejando 20% de nuestro dataset para validar.

Capaz es un poco mucho ese porcentaje pero creo que eso aseguraria que los resultados obtenidos tendran un poco mas de robustez.

Dentro de desarrollo capaz podemos hacer algo como k folds con k = 4 para sacarle un poco mas el jugo a los datos.

train = 80%
validacion = 20%

In [54]:
def train_test_split(df,train_size=0.8):
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    n = df.shape[0]
    cantNeg, cantPos = df['target'].value_counts()
    proporcion = cantPos / n

    cant_en_train = int(n*train_size)
    cant_en_validacion = n - cant_en_train
    
    res = []
    cant_positivas = int(proporcion * cant_en_validacion)
    cant_negativas = cant_en_validacion - cant_positivas

    contador_positivos, contador_negativos = 0, 0
    for index, value in enumerate(df['target']):
        if value == 0:
            contador_negativos += 1
            res.append(index)
        
        if contador_negativos == cant_negativas:
            break
    for index, value in enumerate(df['target']):

        if value == 1:
            contador_positivos += 1
            res.append(index)
        
        if contador_positivos == cant_positivas:
            break
    
    df_dev = df.loc[~df.index.isin(res)].reset_index(drop=True)
    df_val = df.loc[res].reset_index(drop=True)
    return df_dev,df_val

In [53]:
df_dev , df_val =  train_test_split(df)
df_train, df_test = train_test_split(df_dev)

# Ejercicio 2

In [45]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, roc_auc_score , average_precision_score

In [46]:
X_train = df_train.drop(columns=['target']).values

y_train = df_train['target'].values

X_test  = df_test.drop(columns=['target']).values

y_test  = df_test['target'].values

X_val   = df_val.drop(columns=['target']).values

y_val   =  df_val['target'].values

In [47]:
tree_clf = DecisionTreeClassifier(max_depth=3, random_state=51)
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, random_state=51)

In [57]:
scoring = ['accuracy', 'roc_auc', 'average_precision']
skf = StratifiedKFold(n_splits=5)

cv_results_train = cross_validate(tree_clf, X_train, y_train, cv = skf, scoring = scoring)

print("Cross-validation results in Train:")
for metric in scoring:
    print(f"{metric.capitalize()}: {cv_results_train[f'test_{metric}'].round(3)}")
    print(f"Promedio de {metric.capitalize()}:  {cv_results_train[f'test_{metric}'].mean().round(3)}")
    print('\n')

print("--------------------------------------------------------")

cv_results_test = cross_validate(tree_clf, X_test, y_test, cv = skf, scoring = scoring)

print("Cross-validation results in Test:")
for metric in scoring:
    print(f"{metric.capitalize()}: {cv_results_test[f'test_{metric}'].round(3)}")
    print(f"Promedio de {metric.capitalize()}:  {cv_results_test[f'test_{metric}'].mean().round(3)}")
    print('\n')


Cross-validation results in Train:
Accuracy: [0.694 0.653 0.597 0.736 0.625]
Promedio de Accuracy:  0.661


Roc_auc: [0.644 0.614 0.56  0.72  0.579]
Promedio de Roc_auc:  0.623


Average_precision: [0.408 0.456 0.363 0.533 0.396]
Promedio de Average_precision:  0.431


--------------------------------------------------------
Cross-validation results in Test:
Accuracy: [0.722 0.611 0.5   0.667 0.778]
Promedio de Accuracy:  0.656


Roc_auc: [0.492 0.531 0.521 0.667 0.806]
Promedio de Roc_auc:  0.603


Average_precision: [0.333 0.311 0.44  0.433 0.634]
Promedio de Average_precision:  0.43




Basado en pagina 288 de Introduction to Machine Learning with Python

In [60]:
alturas = [3, 5, None]
criterios = ["gini", "entropy"]

resultados = {}

for altura in alturas:
    for criterio in criterios:
        
        tree = DecisionTreeClassifier(max_depth = altura, criterion = criterio, random_state = 51)
        
        tree.fit(X_train, y_train)
        
        cv_results_train = cross_validate(tree, X_train, y_train, cv = skf, scoring = 'accuracy')
        
        y_pred_test = tree.predict(X_test)

        accuracy_test = accuracy_score(y_test, y_pred_test)
        
        global_score_test= cv_results_train['test_score'].mean()
        
        resultados[(altura, criterio)] = (global_score_test,accuracy_test)

results_df = pd.DataFrame(resultados.values(), index=resultados.keys(), columns=['Train Score', 'Test Score'])



In [61]:
results_df

Train Score  Test Score
3   gini        0.661111    0.633333
    entropy     0.638889    0.755556
5   gini        0.644444    0.600000
    entropy     0.650000    0.700000
NaN gini        0.630556    0.577778
    entropy     0.652778    0.700000

# Ejercicio 3

In [64]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

# Define the hyperparameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    #'max_features': ['auto', 'sqrt', 'log2', None]
}

# Create the Decision Tree classifier
tree = DecisionTreeClassifier()

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=tree, param_distributions=param_grid, scoring='roc_auc', cv=5)

# Fit the RandomizedSearchCV object to the data
random_search.fit(X_train, y_train)

# Get the best model and its performance
best_model = random_search.best_estimator_
best_score = random_search.best_score_

# Print the best model and its performance
print("Best Model:", best_model)
print("Best AUCROC Score:", best_score)

Best Model: DecisionTreeClassifier(min_samples_split=5)
Best AUCROC Score: 0.636304347826087


In [66]:
from sklearn.neighbors import KNeighborsClassifier

# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)

# Train the model using the development data
knn.fit(X_train, y_train)

# Predict the target variable for the test data
y_pred_knn = knn.predict(X_test)

# Calculate the accuracy score for KNN
accuracy_knn = accuracy_score(y_test, y_pred_knn)

# Compare the accuracy scores of Decision Tree and KNN
comparison = {'Decision Tree': accuracy_test, 'KNN': accuracy_knn}
comparison


{'Decision Tree': 0.7, 'KNN': 0.8}